In [ ]:
from comprehensive_selection import *

from jqdata import *
import pandas as pd

# 设置截止日期和回溯天数
end_date = '2025-02-18'
lookback_days = 5

# 获取所有交易日
trade_dates = get_trade_days(end_date=end_date, count=lookback_days)
# 转换为字符串格式的日期列表
test_dates = [date.strftime('%Y-%m-%d') for date in trade_dates]

print(f"将测试以下交易日: {test_dates}\n")

write_file("daily_rs/selected_list.txt", "",append=False)
# 对每个交易日运行选股策略
for date in test_dates:
    print(f"\n=== {date} 的选股结果 ===")
    result = run_stock_selection(date)
    #date,result的股票代码转换为一行，用write_file方法在daily_rs/selected_list.txt追加写入
    # 将日期和股票代码组合成一行
    line = f"{date},{','.join(result)}\n"
    # 追加写入到文件
    write_file("daily_rs/selected_list.txt", line, append=True)
    print(f"筛选的股票代码：{result}")


In [ ]:
# 从daily_bottom_finder模块导入所需的函数和类
from candle_stick_ananly import (
    get_low_and_high
)
from jqdata import *
from stock_backtesting import StockBacktesting

# 初始化回测系统
backtester = StockBacktesting(init_cash=500000, max_stock_num=5)

#按照test_date循环执行
for sepc_date in test_dates:
    print(f"\n=== {sepc_date} 交易分析 ===")
    
    # 获取次日交易日期
    next_date = get_trade_days(start_date=sepc_date, count=2)[1]
    # 检查next_date是否超出交易数据范围
    if next_date >= pd.Timestamp.now().date():
        print(f"日期 {next_date} 超过交易数据范围，跳过")
        continue
    
    # 遍历持仓列表检查是否需要卖出
    positions_to_remove = []
    for position in backtester.get_holdings():
        # 获取当日价格数据
        stock_code = position['code']
        target_price = position['target_price']

        daily_data = get_price(stock_code, start_date=next_date, end_date=next_date, 
                             frequency='daily', fields=['open', 'close', 'high', 'low'])
        
        if daily_data is None or daily_data.empty:
            continue
        #更新market_price
        position['market_price'] = daily_data['close'][0]
        
        # 检查是否达到目标价格
        if daily_data['high'][0] >= target_price:
            profit = backtester.sell_stock(stock_code, target_price, sepc_date)
            print(f"\n{stock_code} 达到目标价格 ¥{target_price:.2f}，卖出获利 {profit:.2f}%")
            continue
            
        # 检查是否到达清仓日期
        if position['clear_date'] == next_date:
            open_price = daily_data['open'][0]
            close_price = daily_data['close'][0]
            
            # 根据开盘价决定卖出价格
            if open_price > position['price']:
                sell_price = open_price
            else:
                sell_price = min(position['price'] * 0.98, close_price)
                
            profit = backtester.sell_stock(stock_code, sell_price, sepc_date)
            print(f"\n{stock_code} 到达清仓日期，以 ¥{sell_price:.2f} 卖出，收益率 {profit:.2f}%")

    # 获取当前需要判断是否买入的股票列表
    stock_codes = get_stocks_by_date(sepc_date)
    
    # 获取当日股票数据
    df = get_low_and_high(sepc_date, stock_codes)
    
    for stock in stock_codes:
        # 检查股票是否已在持仓中
        if backtester.is_stock_in_holdings(stock):
            print(f"{stock} 已在持仓中，跳过")
            continue
            
        buy_price = df.loc[stock].la
        
        # 获取次日最低价
        next_day_data = get_price(stock, start_date=next_date, end_date=next_date, 
                                 frequency='daily', fields=['low','high'])
        next_day_low = next_day_data['low'][0]
        next_day_high = next_day_data['high'][0]
        #目标价：target_price = round(max(buy_price*1.05, next_day_high), 2)
        target_price = max(buy_price * 1.05, next_day_high)
        
        #设置平仓日期为5个交易日后
        clear_date = get_trade_days(start_date=next_date, count=6)[-1]
        #执行买入操作
        backtester.buy_stock(stock, buy_price, clear_date, target_price)
    
    #输出当前持仓情况
    print("\n=== 持仓情况 ===")
    holdings = backtester.get_holdings()
    for position in holdings:
        print(f"股票代码: {position['code']}, 买入价: ¥{position['price']:.2f}, 市场价: ¥{position['market_price']:.2f}, 目标价: ¥{position['target_price']:.2f}, 平仓日期: {position['clear_date']},持仓收益率: {(position['market_price']-position['price'])/position['price']:.2f}%")

    
    # 输出当前账户状态
    print("\n当前账户状态:")
    print(backtester.summary_status())


# ... existing code ...

: 